AMT5005MX Python Project AppleOrangeAI

Installs PIP necessary for code

In [1]:
%pip install torch
%pip install torchvision
%pip install matplotlib
%pip install scikit-learn
%pip install tensorboard
%pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Necessary Pytorch Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

Adjustable Hyperparamaters

In [3]:
num_classes = 2 # DO NOT CHANGE ‘num_classes’: The number of classes in our new classification task.
batch_size = 32 #- ‘batch_size’: Number of images processed at a time. A typical value is 32 or 64.
num_epochs = 1 #- ‘num_epochs’: Number of complete passes through the training dataset. Here we use 1 for demonstration, but in practice, you may use 10, 20, or more.
learning_rate = 0.001 #- ‘learning_rate’: The rate at which the model updates its weights during training.

Data Preprocessing

In [4]:
data_transforms = {
 'train': transforms.Compose([
 transforms.Resize((224, 224)), #all images are resized to VVG16, fixing outliers and making the images optimal for VGG16
 transforms.RandomHorizontalFlip(), #Randomly flips the images horizontally, this effectively allows for more training and test material by doubling the number of available images (flipped and unflipped)
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #Normalizes the pixel values (mandatory for using my pre-trained model)
 ]),
 'val': transforms.Compose([
 transforms.Resize((224, 224)),
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ]),
}

Root/ Relative Paths

In [5]:
#Relative path based off the name of the files, change if renamed/moved
root_dir = './apple_orange'
train_root = root_dir + '/train'
val_root = root_dir + '/val'

In [6]:
#Loading the training and validation datasetts using 'ImageFolder' for later use
train_dataset = datasets.ImageFolder(root=train_root, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(root=val_root, transform=data_transforms['val'])

In [7]:
#creates data loaders to fectch data in batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) 
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
print(train_dataset.classes) #Print class names, should be ['apple', 'orange']

['apple', 'orange']


Reading CSV files using Pandas

In [8]:
#imports pandas and prints the current pandas version
import pandas
import pandas as pd

In [9]:
#input CSV file
dataFrame = pd.read_csv('./apple_orange/metadata.csv')
#Check CSV file is loaded correctly
print("Our Dataframe....", dataFrame)

Our Dataframe....              image_id      domain  split                  image_path
0     n07740461_10011  B (Orange)   test   testB/n07740461_10011.jpg
1     n07740461_10012   A (Apple)  train  trainA/n07740461_10012.jpg
2     n07740461_10019   A (Apple)  train  trainA/n07740461_10019.jpg
3     n07740461_10037   A (Apple)  train  trainA/n07740461_10037.jpg
4     n07740461_10065   A (Apple)  train  trainA/n07740461_10065.jpg
...               ...         ...    ...                         ...
2523   n07749192_9883  B (Orange)  train   trainB/n07749192_9883.jpg
2524   n07749192_9889  B (Orange)  train   trainB/n07749192_9889.jpg
2525    n07749192_992  B (Orange)  train    trainB/n07749192_992.jpg
2526    n07749192_993  B (Orange)  train    trainB/n07749192_993.jpg
2527    n07749192_998  B (Orange)  train    trainB/n07749192_998.jpg

[2528 rows x 4 columns]


Loading Pretrained Model

In [10]:
#loads the Pre-Trained VGG16 Model
model = models.vgg16(pretrained=True)

c:\Users\Myles\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Myles\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model Adjustment

In [11]:
#Freezes unnecessary parts of the model to save resources
for param in model.features.parameters():
 param.requires_grad = False

In [12]:
#modifies the final layer to match the number of classes in our task
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

In [13]:
#creates a Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

Model Finetuning (Takes a long time)

In [14]:
# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

Using device: cpu


In [15]:
for epoch in range(num_epochs):
   model.train()
   running_loss = 0.0
   for inputs, labels in train_loader:
     inputs, labels = inputs.to(device), labels.to(device)
     optimizer.zero_grad()
     outputs = model(inputs)
     loss = criterion(outputs, labels)
     loss.backward()
     optimizer.step()
     running_loss += loss.item() * inputs.size(0)
   epoch_loss = running_loss / len(train_loader.dataset)
   print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/1, Loss: 0.6742


Model Evaluation (Takes a long time)

In [16]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f'Validation Accuracy: {val_accuracy:.4f}')

Validation Accuracy: 0.9689


Saving the trained model for later use

In [17]:
#saves the trained model
torch.save(model.state_dict(), 'vgg16_finetuned.pth')

In [18]:
# Load the saved state dictionary
state_dict = torch.load('vgg16_finetuned.pth')

# Load the state dictionary into the model
model.load_state_dict(state_dict)

<All keys matched successfully>

Reading CSV files using Pandas

In [19]:
#imports pandas and prints the current pandas version
import pandas
import pandas as pd

In [20]:
#input CSV file
dataFrame = pd.read_csv('./apple_orange/metadata.csv')
#Check CSV file is loaded correctly
print("Our Dataframe....", dataFrame)

Our Dataframe....              image_id      domain  split                  image_path
0     n07740461_10011  B (Orange)   test   testB/n07740461_10011.jpg
1     n07740461_10012   A (Apple)  train  trainA/n07740461_10012.jpg
2     n07740461_10019   A (Apple)  train  trainA/n07740461_10019.jpg
3     n07740461_10037   A (Apple)  train  trainA/n07740461_10037.jpg
4     n07740461_10065   A (Apple)  train  trainA/n07740461_10065.jpg
...               ...         ...    ...                         ...
2523   n07749192_9883  B (Orange)  train   trainB/n07749192_9883.jpg
2524   n07749192_9889  B (Orange)  train   trainB/n07749192_9889.jpg
2525    n07749192_992  B (Orange)  train    trainB/n07749192_992.jpg
2526    n07749192_993  B (Orange)  train    trainB/n07749192_993.jpg
2527    n07749192_998  B (Orange)  train    trainB/n07749192_998.jpg

[2528 rows x 4 columns]


Confusion Matrix

In [21]:
#Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

from torch.utils.tensorboard import SummaryWriter

In [22]:
import itertools
import numpy as np

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [23]:
cm = confusion_matrix(test_set.targets, test_preds.argmax(dim=1))
classes = ('apples', 'oranges')
plt.figure(figsize=(2,2))
plot_confusion_matrix(cm, classes)

NameError: name 'test_set' is not defined

In [ ]:
cm = confusion_matrix(test_set.targets, test_preds.argmax(dim=1))
classes = ('apples', 'oranges')
plt.figure(figsize=(2,2))
plot_confusion_matrix(cm, classes)

If you can the training loss curve also, something like attached in the diagram.